<h1>ETL</h1>

In [123]:
# importando as bibliotecas usadas no projeto

import pandas as pd
import requests 
from plyer import notification
from pandas import json_normalize
import sqlite3


In [124]:

def get_data(url):

    '''
    Faz a requisição à API. Se o status code for 200, retorna um DataFrame pandas.
    Se não, dispara uma notificação de erro e retorna None.
    
    Parâmetros:
      - url (str): A URL da API.
      
    Retorna:
      - DataFrame pandas ou None em caso de erro.
    '''
    
    response = requests.get(url)

    if response.status_code == 200:
        data_json = response.json()
        data = pd.DataFrame(data_json)
        return data
    
    else:
       
        notification_title = "Erro na solicitação da API"
        notification_text = f"O request retornou o status {response.status_code}. Verifique a URL."
        
        notification.notify(
            title=notification_title,
            message=notification_text,
        )
        return None 


In [125]:
# informações dos paises com a api restcountries já pegando as colunas nos endpoints

countries_df1 = get_data("https://restcountries.com/v3.1/all?fields=name,capital,region,population")
countries_df2 = get_data("https://restcountries.com/v3.1/all?fields=name,subregion,continents")
countries_df3 = get_data("https://restcountries.com/v3.1/all?fields=name,timezones,startOfWeek")


In [126]:
def normalize_columns(dataframe,columns_data):

    '''
    Normaliza as colunas especificadas em um DataFrame.

    Parâmetros:
      - dataframe: DataFrame pandas
      - columns_data: Lista de nomes de colunas a serem normalizadas

    Retorna:
      - DataFrame normalizado

    '''
    
    for coluna in columns_data:
        dataframe = pd.concat([dataframe, json_normalize(dataframe[coluna])], axis=1)

   
    return dataframe

In [127]:

#aplica a função normalize_columns ao dataframe e seleciona as colunas de interece.
  
countries_df1 = normalize_columns(countries_df1, ['name']).loc[:,['common','region','population']]

In [128]:
#aplica a função normalize_columns ao dataframe e seleciona as colunas de interece.

countries_df2 = normalize_columns(countries_df2, ['name']).loc[:,['common','continents','subregion']]

In [129]:
#aplica a função normalize_columns ao dataframe e seleciona as colunas de interece.

countries_df3 = normalize_columns(countries_df3, ['name']).loc[:,['common','timezones','startOfWeek']]

In [130]:
#faz a junção dos dataframes em um unico dataframe.

countries = pd.merge(pd.merge(countries_df1, countries_df2, on='common'), countries_df3, on='common')


In [131]:
# faz a mudança de nome das colunas do dataframe.

new_columns_name = {'common': 'Pais','region':'Região','population':'População','continents':'Continentes','subregion':'Subregião',
                    'timezones':'Zonas Horarias','startOfWeek':'Primeiro dia da semana'}

countries.rename(columns=new_columns_name, inplace=True)


In [132]:
columns = ['Continentes', 'Zonas Horarias']

# Itera sobre cada coluna a ser processada

for coluna in columns:
    
    # Aplica uma função lambda para extrair colchetes

    countries[coluna] = countries[coluna].apply(lambda x: x[0] if isinstance(x, list) else x)


In [133]:
# cria a conexão com o banco de dados.

conn =sqlite3.connect('coutries.db')

countries.to_sql('countries',conn, index=False,if_exists='replace')

conn.close()

In [134]:
countries = pd.DataFrame(countries)

In [135]:
countries

,Pais,Região,População,Continentes,Subregião,Zonas Horarias,Primeiro dia da semana
0,Christmas Island,Oceania,2072,Asia,Australia and New Zealand,UTC+07:00,monday
1,Eritrea,Africa,5352000,Africa,Eastern Africa,UTC+03:00,monday
2,Samoa,Oceania,198410,Oceania,Polynesia,UTC+13:00,monday
3,North Macedonia,Europe,2077132,Europe,Southeast Europe,UTC+01:00,monday
4,Djibouti,Africa,988002,Africa,Eastern Africa,UTC+03:00,monday
...,...,...,...,...,...,...,...
245,Azerbaijan,Asia,10110116,Europe,Western Asia,UTC+04:00,monday
246,Timor-Leste,Asia,1318442,Oceania,South-Eastern Asia,UTC+09:00,monday
247,Iran,Asia,83992953,Asia,Southern Asia,UTC+03:30,saturday
248,Croatia,Europe,4047200,Europe,Southeast Europe,UTC+01:00,monday


In [136]:
countries.stack()

0    Pais                               Christmas Island
     Região                                      Oceania
     População                                      2072
     Continentes                                    Asia
     Subregião                 Australia and New Zealand
                                         ...            
249  População                                  26378275
     Continentes                                  Africa
     Subregião                            Western Africa
     Zonas Horarias                                  UTC
     Primeiro dia da semana                       monday
Length: 1750, dtype: object

In [137]:
countries.shape

(250, 7)

In [138]:
countries.nunique()

Pais                      250
Região                      6
População                 248
Continentes                 7
Subregião                  25
Zonas Horarias             31
Primeiro dia da semana      3
dtype: int64

In [139]:
countries.isnull().sum()

Pais                      0
Região                    0
População                 0
Continentes               0
Subregião                 0
Zonas Horarias            0
Primeiro dia da semana    0
dtype: int64